# Testing basic GCN on our modified real-world tasks

In [1]:
import torch
import torch_geometric

from datasets import get_dblp_tasks, get_elliptic_temporal_tasks, get_ogbn_arxiv_tasks, get_dblp, get_ogbn_arxiv

In [2]:
#dblp = get_dblp_tasks()
#elliptic = get_elliptic_temporal_tasks()
#ogbn = get_ogbn_arxiv_tasks()
ogbn = get_ogbn_arxiv_tasks()

D:\Users\Productivity\anaconda3\envs\cudanew\lib\site-packages\torch_geometric\data\in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [3]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.conv1 = GCNConv(input_dim, 128)
        self.conv2 = GCNConv(128, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [4]:
from measures import Result
from datasets import get_mask

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.set_printoptions(precision=3)

# dblp_result = Result(dblp, GCN(len(dblp[0].x[0]), 44), device)
# dblp_result.learn()
# print(f'DBLP\tAP:{dblp_result.get_average_accuracy():.2f}\tAF:{dblp_result.get_average_forgetting_measure():.2f}\n')
# print(dblp_result.get_result_matrix())

# elliptic_result = Result(elliptic, GCN(len(elliptic[0].x[0]), 2), device)
# elliptic_result.learn()
# print(f'Elliptic\tAP:{elliptic_result.get_average_accuracy():.2f}\tAF:{elliptic_result.get_average_forgetting_measure():.3f}\n')
# print(elliptic_result.get_result_matrix())

ogbn_result = Result(ogbn, GCN(len(ogbn[0].x[0]), len(torch.unique(ogbn[0].y))), device)

cuda


In [6]:
ogbn[1].x.size()

torch.Size([23836, 128])

In [7]:
ogbn[0].y.size()

torch.Size([17401])

In [8]:
ogbn[0].train_mask.sum()

tensor(10440)

In [9]:
ogbn_result.learn()

In [10]:
ogbn_result.get_result_matrix()

tensor([[0.568, 0.502, 0.508, 0.491, 0.482, 0.447, 0.453, 0.433, 0.403, 0.388],
        [0.523, 0.554, 0.530, 0.507, 0.489, 0.464, 0.463, 0.459, 0.444, 0.431],
        [0.520, 0.524, 0.557, 0.524, 0.520, 0.504, 0.509, 0.500, 0.470, 0.444],
        [0.512, 0.520, 0.522, 0.553, 0.531, 0.520, 0.524, 0.522, 0.497, 0.482],
        [0.517, 0.507, 0.509, 0.540, 0.552, 0.534, 0.544, 0.545, 0.518, 0.504],
        [0.506, 0.490, 0.498, 0.533, 0.531, 0.555, 0.552, 0.557, 0.536, 0.528],
        [0.498, 0.484, 0.493, 0.525, 0.525, 0.541, 0.563, 0.561, 0.540, 0.529],
        [0.482, 0.471, 0.470, 0.508, 0.509, 0.530, 0.550, 0.569, 0.563, 0.541],
        [0.463, 0.458, 0.441, 0.494, 0.495, 0.506, 0.541, 0.553, 0.571, 0.550],
        [0.455, 0.444, 0.433, 0.454, 0.474, 0.485, 0.510, 0.530, 0.555, 0.546]])

In [11]:
ogbn_result.get_average_forgetting_measure()

0.0785408185587989